In [1]:
import os, sys
cwd = os.getcwd()
module_path = os.path.abspath(os.path.join('..'))
student_life_path = module_path + "/student_life"
physnet_path = module_path + "/Physionet_rev"
print(student_life_path)
print(physnet_path)

sys.path.append(physnet_path)
sys.path.append(student_life_path)

import pandas as pd
import numpy as np
import pickle
from random import shuffle
from tqdm import tqdm
import matplotlib.pyplot as plt
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd


import tbm as tbm
from net_dbm import RNN_osaka


import evaluate_plot as eval_plot
import batchify as batchify
from sklearn.metrics import precision_recall_fscore_support
import importlib
import src.utils.student_utils as student_utils
import src.definitions as definitions
from IPython.display import display
from src.data_getter import student_life_var_binned_data_getter
importlib.reload(student_life_var_binned_data_getter)
importlib.reload(student_utils)

# 1 starts the process on GPU-0
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
torch.__version__
%matplotlib inline
print(eval_plot)
print(sys.version_info)

/Users/nsimsiri/Documents/code/ml/MultiRes/student_life
/Users/nsimsiri/Documents/code/ml/MultiRes/Physionet_rev
<module 'evaluate_plot' from '/Users/nsimsiri/Documents/code/ml/MultiRes/Physionet_rev/evaluate_plot.py'>
sys.version_info(major=3, minor=6, micro=0, releaselevel='final', serial=0)


In [2]:
data = None
pickle_path = student_life_path + '/data/training_data/student_life_pickle.pkl'
model_path = student_life_path + '/models'
print('pickle_path', pickle_path)
print('model_config_path', model_path)

pickle_path /Users/nsimsiri/Documents/code/ml/MultiRes/student_life/data/training_data/student_life_pickle.pkl
model_config_path /Users/nsimsiri/Documents/code/ml/MultiRes/student_life/models


In [3]:
with open(pickle_path, 'rb') as pickle_file:
    data = pickle.load(pickle_file)
print(data.keys())

dict_keys(['train_ids', 'val_ids', 'test_ids', 'data'])


In [4]:
params = {'bilstm_flag':True,
        'hidden_dim' : 128,
        'input_dim':50,
        'dropout' : 0.9,
        'layers' : 1,
        'tagset_size' : 5,
        'bilstm_flag' : True,
        'attn_category' : 'dot',
        'num_features' : 10,
        'batch_size':1,
        'model_name':'TBM-SL-'}

model_config_path = model_path + "/" + params['model_name']+'.pt'
print(model_config_path)
with open(model_config_path, 'wb') as config_file:
    pickle.dump(params, config_file)
    


/Users/nsimsiri/Documents/code/ml/MultiRes/student_life/models/TBM-SL-.pt


In [5]:
model_RNN = tbm.RNN_osaka(params)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model_RNN.parameters(), lr=0.0001, weight_decay=0.000000002)


Dot Attention is being used!


/Users/nsimsiri/anaconda2/envs/python3/lib/python3.6/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.9 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [6]:
mode = 'normal'
if(mode=='normal'):
    feature_ind = 0
    label_ind = -1
    print("NORMAL mode with Flags")

NORMAL mode with Flags


In [7]:
batch_size = 1
epochs = 45
save_flag = True
dict_df_prf_mod = {}
print("==x=="*20)
print("Data Statistics")
print("Train Data: "+str(len(data['train_ids'])))
print("Val Data: "+str(len(data['val_ids'])))
print("Test Data: "+str(len(data['test_ids'])))
print("==x=="*20)

==x====x====x====x====x====x====x====x====x====x====x====x====x====x====x====x====x====x====x====x==
Data Statistics
Train Data: 17
Val Data: 5
Test Data: 7
==x====x====x====x====x====x====x====x====x====x====x====x====x====x====x====x====x====x====x====x==


In [8]:
start_epoch = 0
end_epoch = 90
model_name = params['model_name']
target_classes = [0,1,2,3,4]
for iter_ in range(start_epoch, end_epoch):
    print ("=#="*5+str(iter_)+"=#="*5)
    total_loss = 0
    preds_train = []
    actual_train = []
    for each_ID in tqdm(data['train_ids']):
        model_RNN.zero_grad()
        tag_scores = model_RNN(data['data'], each_ID)
        
        _, ind_ = torch.max(tag_scores, dim=1)
        preds_train+=ind_.tolist()
        
        curr_labels = [data['data'][each_ID][-1]]
        actual_train+=curr_labels
        curr_labels = torch.LongTensor(curr_labels)
        curr_labels = autograd.Variable(curr_labels)
        
        loss = loss_function(tag_scores, curr_labels.reshape(tag_scores.shape[0]))
        total_loss+=loss.item()
        loss.backward()
        optimizer.step()
    
    df_tr = pd.DataFrame(list(precision_recall_fscore_support(actual_train, preds_train, labels = target_classes)),
                       columns = list(range(len(target_classes))))
    df_tr.index = ['Precision','Recall','F-score','Count']
    prf_tr = precision_recall_fscore_support(actual_train, preds_train, average='weighted')
#     prf_test, df_test = eval_plot.evaluate_dbm(model_RNN, data, 'test_ids', target_n=target_classes)

#     prf_val, df_val = eval_plot.evaluate_dbm(model_RNN, data, 'val_ids', target_n=target_classes)
    
#     df_all = pd.concat([df_tr, df_val, df_test],axis=1)
#     dict_df_prf_mod['Epoch'+str(iter_)] = df_all
    
    print('=='*5 + "Epoch No:"+str(iter_) +"=="*5)
    print("Training Loss: "+str(total_loss))
    print("=="*4)
    print("Train: " + str(prf_tr))
    print(df_tr)
#     print("--"*4)
#     print("Val: " + str(prf_val))
#     print(df_val)
#     print("--"*4)
#     print("Test: " + str(prf_test))
#     print(df_test)
#     print('=='*40)
#     print('\n')

#     eval_plot.plot_graphs(dict_df_prf_mod, 'F-score', 
#                   model_path + "/" +model_name+str(iter_)+'.png',
#                   0, iter_+1, 
#                   model_name, target_n=len(target_classes))

#     if(save_flag):
#         torch.save(model_RNN, '../../Models/'+model_name+str(iter_)+'.pt')
#         pickle.dump(dict_df_prf_mod, open('../../Results/dict_prf_'+model_name+str(iter_)+'.pkl','wb'))
#         eval_plot.plot_graphs(dict_df_prf_mod, 'F-score', 
#                               '../../Plots/'+model_name+str(iter_)+'.png',
#                               0, iter_+1, 
#                               model_name)

  0%|          | 0/17 [00:00<?, ?it/s]

=#==#==#==#==#=0=#==#==#==#==#=


100%|██████████| 17/17 [00:34<00:00,  1.91s/it]
/Users/nsimsiri/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/nsimsiri/anaconda2/envs/python3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
  0%|          | 0/17 [00:00<?, ?it/s]

==========Epoch No:0==========
Training Loss: 27.607382774353027
Train: (0.1497326203208556, 0.23529411764705882, 0.1830065359477124, None)
             0    1         2    3    4
Precision  0.0  0.0  0.363636  0.0  0.0
Recall     0.0  0.0  0.571429  0.0  0.0
F-score    0.0  0.0  0.444444  0.0  0.0
Count      3.0  6.0  7.000000  0.0  1.0
=#==#==#==#==#=1=#==#==#==#==#=


  0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/17 [00:00<?, ?it/s]

==========Epoch No:16==========
Training Loss: 24.201445817947388
Train: (0.4973262032085561, 0.6470588235294118, 0.5555555555555556, None)
             0         1         2    3    4
Precision  0.0  0.666667  0.636364  0.0  0.0
Recall     0.0  0.666667  1.000000  0.0  0.0
F-score    0.0  0.666667  0.777778  0.0  0.0
Count      3.0  6.000000  7.000000  0.0  1.0
=#==#==#==#==#=17=#==#==#==#==#=


  0%|          | 0/17 [00:00<?, ?it/s]

==========Epoch No:17==========
Training Loss: 24.089420676231384
Train: (0.4973262032085561, 0.6470588235294118, 0.5555555555555556, None)
             0         1         2    3    4
Precision  0.0  0.666667  0.636364  0.0  0.0
Recall     0.0  0.666667  1.000000  0.0  0.0
F-score    0.0  0.666667  0.777778  0.0  0.0
Count      3.0  6.000000  7.000000  0.0  1.0
=#==#==#==#==#=18=#==#==#==#==#=


KeyboardInterrupt: 

In [ ]:
print(data)